In [9]:
!pip uninstall -y pyarrow datasets

Found existing installation: pyarrow 16.0.0
Uninstalling pyarrow-16.0.0:
  Successfully uninstalled pyarrow-16.0.0
Found existing installation: datasets 2.19.0
Uninstalling datasets-2.19.0:
  Successfully uninstalled datasets-2.19.0


In [10]:
!pip install --no-use-pep517 pyarrow

  Using cached pyarrow-16.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
Using cached pyarrow-16.0.0-cp311-cp311-manylinux_2_28_x86_64.whl (40.8 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlflow 2.11.3 requires pyarrow<16,>=4.0.0, but you have pyarrow 16.0.0 which is incompatible.


In [11]:
!pip install datasets

  Using cached datasets-2.19.0-py3-none-any.whl.metadata (19 kB)
Using cached datasets-2.19.0-py3-none-any.whl (542 kB)


In [12]:
from datasets import load_dataset

dataset = load_dataset("Hello-SimpleAI/HC3", name="all")

In [16]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'human_answers', 'chatgpt_answers', 'source'],
        num_rows: 24322
    })
})
